In [3]:
"""
Dashboard Web Interactif LCRPGR - Rwanda (Version 3 propre)
Données attendues dans : Data/
- population_2015.tif
- population_2025.tif
- built_up_2015.tif
- built_up_2025.tif
- gadm41_RWA_0.shp (frontières)
- gadm41_RWA_1.shp (provinces)
- gadm41_RWA_2.shp (districts)
- gadm41_RWA_3.shp (secteurs)
"""

import os
import json
import numpy as np
import pandas as pd

import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask

import geopandas as gpd

import dash
from dash import dcc, html, Input, Output, State, dash_table
import plotly.graph_objects as go
import plotly.express as px


# ------------------------------------------------------------
# 1) CONFIG / CHEMINS
# ------------------------------------------------------------
DATA_DIR = "Data"

def _path_raster(base_name):
    # accepte: population_2015 (=> population_2015.tif)
    candidates = [
        os.path.join(DATA_DIR, f"{base_name}.tif"),
        os.path.join(DATA_DIR, f"{base_name}.tiff"),
        os.path.join(DATA_DIR, f"{base_name}.img")
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Raster introuvable pour '{base_name}' dans {DATA_DIR} (attendu .tif/.tiff/.img)")

def _path_shp(base_name):
    p = os.path.join(DATA_DIR, f"{base_name}.shp")
    if not os.path.exists(p):
        raise FileNotFoundError(f"Shapefile introuvable: {p}")
    return p


# ------------------------------------------------------------
# 2) CHARGEMENT DES DONNÉES (RASTERS + SHAPEFILES)
# ------------------------------------------------------------
print("\n📂 Chargement des données...")

# Rasters (population + built-up)
pop_2015 = rasterio.open(_path_raster("population_2015"))
pop_2025 = rasterio.open(_path_raster("population_2025"))
built_2015 = rasterio.open(_path_raster("built_up_2015"))
built_2025 = rasterio.open(_path_raster("built_up_2025"))

# Frontières nationales (niveau 0) + niveaux admin (1/2/3)
rwanda = gpd.read_file(_path_shp("rwanda_boundary"))
gadm1 = gpd.read_file(_path_shp("gadm41_RWA_1"))
gadm2 = gpd.read_file(_path_shp("gadm41_RWA_2"))
gadm3 = gpd.read_file(_path_shp("gadm41_RWA_3"))

# Harmonisation CRS (important)
target_crs = pop_2015.crs
rwanda = rwanda.to_crs(target_crs)
gadm1 = gadm1.to_crs(target_crs)
gadm2 = gadm2.to_crs(target_crs)
gadm3 = gadm3.to_crs(target_crs)

admin_data = {
    "provinces": gadm1,
    "districts": gadm2,
    "sectors": gadm3
}

print("✅ Données chargées (rasters + shapefiles).")


# ------------------------------------------------------------
# 3) PRÉPARATION RASTERS (numpy) + INDICATEURS PIXEL
# ------------------------------------------------------------
def _clean(arr):
    arr = arr.astype(float).copy()
    arr[arr < 0] = np.nan
    arr[arr == -99999] = np.nan
    return arr

# Lecture des rasters population (déjà alignés si mêmes tuiles/CRS)
pop2015 = _clean(pop_2015.read(1))
pop2025 = _clean(pop_2025.read(1))

# built-up: on masque d’abord au Rwanda (au cas où le raster déborde)
rwa_geoms = rwanda.geometry.values
built2015_masked, _ = mask(built_2015, rwa_geoms, crop=True)
built2025_masked, _ = mask(built_2025, rwa_geoms, crop=True)

# On recadre built pour avoir la même forme que pop (si nécessaire)
def _crop_to_shape(data, shape):
    h, w = shape
    return data[:h, :w]

built2015 = _clean(_crop_to_shape(built2015_masked[0], pop2015.shape))
built2025 = _clean(_crop_to_shape(built2025_masked[0], pop2015.shape))

# éviter les divisions par zéro sur built-up
built2015[built2015 == 0] = 0.001
built2025[built2025 == 0] = 0.001

T = 10  # période 2015 -> 2025 (années)

# Croissance annuelle (en %/an)
pop_growth = (np.log(pop2025 / pop2015) / T) * 100.0
urban_growth = (((built2025 - built2015) / built2015) / T) * 100.0

# LCRPGR = (croissance urbaine) / (croissance population)
LCRPGR = (urban_growth / 100.0) / (pop_growth / 100.0)
LCRPGR[np.isinf(LCRPGR)] = np.nan
LCRPGR = np.clip(LCRPGR, 0.1, 10)

# Densité 2025 (approximation par cellule)
# (ta formule dépend de la résolution ; on la garde telle quelle pour cohérence)
area_pixel_km2 = (0.00833 * 111) ** 2
density_2025 = pop2025 / area_pixel_km2

# Coordonnées pour la visualisation raster (heatmap)
transform = pop_2015.transform
height, width = pop2015.shape
cols, rows = np.meshgrid(np.arange(width), np.arange(height))
lons = transform[2] + cols * transform[0]
lats = transform[5] + rows * transform[4]


# ------------------------------------------------------------
# 4) STATISTIQUES PAR NIVEAU ADMIN (ZONAL STATS)
# ------------------------------------------------------------
def _admin_name_col(level_key: str) -> str:
    return {
        "provinces": "NAME_1",
        "districts": "NAME_2",
        "sectors": "NAME_3"
    }[level_key]

def calculate_admin_stats(gdf: gpd.GeoDataFrame, level_key: str) -> pd.DataFrame:
    name_col = _admin_name_col(level_key)
    rows = []

    for _, r in gdf.iterrows():
        geom = [r.geometry]

        try:
            # masquage raster dans l'entité
            p15_mask, _ = mask(pop_2015, geom, crop=True, all_touched=True)
            p25_mask, _ = mask(pop_2025, geom, crop=True, all_touched=True)
            b15_mask, _ = mask(built_2015, geom, crop=True, all_touched=True)
            b25_mask, _ = mask(built_2025, geom, crop=True, all_touched=True)

            p15 = _clean(p15_mask[0])
            p25 = _clean(p25_mask[0])
            b15 = _clean(b15_mask[0])
            b25 = _clean(b25_mask[0])

            # éviter /0 sur built-up
            b15[b15 == 0] = 0.001
            b25[b25 == 0] = 0.001

            pop15_sum = np.nansum(p15)
            pop25_sum = np.nansum(p25)
            built15_sum = np.nansum(b15)
            built25_sum = np.nansum(b25)

            if pop15_sum <= 0 or pop25_sum <= 0:
                continue

            pop_growth_ent = (np.log(pop25_sum / pop15_sum) / T) * 100.0
            built_growth_ent = (((built25_sum - built15_sum) / built15_sum) / T) * 100.0

            # LCRPGR (éviter division par 0)
            if pop_growth_ent == 0 or np.isnan(pop_growth_ent):
                lcrpgr_ent = np.nan
            else:
                lcrpgr_ent = (built_growth_ent / 100.0) / (pop_growth_ent / 100.0)

            # densité (approx) : surface géométrie (en degrés²) * facteur pour km² (approx)
            area_km2 = r.geometry.area * 12321
            density_ent = pop25_sum / area_km2 if area_km2 > 0 else np.nan

            rows.append({
                "name": r[name_col],
                "pop_2015": pop15_sum,
                "pop_2025": pop25_sum,
                "pop_growth": pop_growth_ent,
                "built_2015": built15_sum,
                "built_2025": built25_sum,
                "built_growth": built_growth_ent,
                "lcrpgr": lcrpgr_ent,
                "density": density_ent
            })

        except Exception as e:
            # on passe silencieusement (ne bloque pas le calcul global)
            continue

    return pd.DataFrame(rows)

admin_stats = {lvl: calculate_admin_stats(gdf, lvl) for lvl, gdf in admin_data.items()}


# ------------------------------------------------------------
# 5) STATISTIQUES NATIONALES (KPI)
# ------------------------------------------------------------
stats = {
    "pop_2015": float(np.nansum(pop2015)),
    "pop_2025": float(np.nansum(pop2025)),
    "pop_growth": float((np.nansum(pop2025) / np.nansum(pop2015) - 1) * 100),
    "built_2015": float(np.nansum(built2015)),
    "built_2025": float(np.nansum(built2025)),
    "built_growth": float((np.nansum(built2025) / np.nansum(built2015) - 1) * 100),
    "lcrpgr_median": float(np.nanmedian(LCRPGR)),
    "density_mean": float(np.nanmean(density_2025))
}


# ------------------------------------------------------------
# 6) DASHBOARD (THEME RWANDA : bleu / jaune / vert / noir)
# ------------------------------------------------------------
app = dash.Dash(__name__, title="LCRPGR Dashboard - Rwanda")

app.index_string = """
<!DOCTYPE html>
<html>
<head>
  {%metas%}
  <title>{%title%}</title>
  {%favicon%}
  {%css%}
  <style>
    body{margin:0;padding:24px;font-family:Segoe UI, Arial, sans-serif;
         background: linear-gradient(135deg, #0f3a6b 0%, #f6c000 70%, #1b5e20 100%);}
    .container{max-width: 1920px; margin: 0 auto; background: #ffffff; border-radius: 18px;
               box-shadow: 0 22px 60px rgba(0,0,0,0.22); padding: 24px;}
    .hero{border-radius: 14px; padding: 16px; color: #ffffff;
          background: linear-gradient(90deg, #0f3a6b 0%, #1b5e20 60%, #000000 100%);}
    .kpi{background: linear-gradient(135deg, #0f3a6b 0%, #f6c000 100%); color:#000;
         border-radius: 14px; padding: 14px; text-align: center;
         box-shadow: 0 8px 22px rgba(0,0,0,0.18);}
    .kpi .v{font-size: 24px; font-weight: 800;}
    .title{color:#0f3a6b; font-weight: 900;}
  </style>
</head>
<body>
  {%app_entry%}
  <footer>{%config%}{%scripts%}{%renderer%}</footer>
</body>
</html>
"""

app.layout = html.Div(className="container", children=[
    html.Div(className="hero", children=[
        html.H1("🇷🇼 Dashboard LCRPGR – Rwanda (2015–2025)", style={"margin": 0}),
        html.P("Population • Urbanisation • LCRPGR — multi-niveaux (Provinces/Districts/Secteurs)",
               style={"margin": "6px 0 0 0"})
    ]),

    # KPI
    html.Div(style={"display": "grid", "gridTemplateColumns": "repeat(4, 1fr)", "gap": "14px", "marginTop": "14px"},
             children=[
                 html.Div(className="kpi", children=[
                     html.Div("Population 2025"),
                     html.Div(f"{stats['pop_2025']:,.0f}", className="v"),
                     html.Div(f"+{stats['pop_growth']:.1f}% depuis 2015", style={"fontSize": 12})
                 ]),
                 html.Div(className="kpi", children=[
                     html.Div("Surface bâtie 2025 (en m²)"),
                     html.Div(f"{stats['built_2025']:,.0f}", className="v"),
                     html.Div(f"+{stats['built_growth']:.1f}% depuis 2015", style={"fontSize": 12})
                 ]),
                 html.Div(className="kpi", children=[
                     html.Div("LCRPGR (médian)"),
                     html.Div(f"{stats['lcrpgr_median']:.2f}", className="v"),
                     html.Div(">1 = étalement", style={"fontSize": 12})
                 ]),
                 html.Div(className="kpi", children=[
                     html.Div("Densité moyenne 2025"),
                     html.Div(f"{stats['density_mean']:.0f}", className="v"),
                     html.Div("hab/km²", style={"fontSize": 12})
                 ]),
             ]),

    # contrôles
    html.Div(style={"display": "grid", "gridTemplateColumns": "1fr 1fr 1fr", "gap": "14px", "marginTop": "18px"}, children=[
        html.Div([
            html.Label("Niveau administratif", style={"fontWeight": "700"}),
            dcc.Dropdown(
                id="admin-level-dropdown",
                options=[
                    {"label": "🇷🇼 National (raster)", "value": "national"},
                    {"label": "🏛️ Provinces", "value": "provinces"},
                    {"label": "🏢 Districts", "value": "districts"},
                    {"label": "🏘️ Secteurs", "value": "sectors"},
                ],
                value="national",
                clearable=False
            )
        ]),
        html.Div([
            html.Label("Indicateur", style={"fontWeight": "700"}),
            dcc.Dropdown(
                id="indicator-dropdown",
                options=[
                    {"label": "👥 Population 2025", "value": "pop_2025"},
                    {"label": "👥 Population 2015", "value": "pop_2015"},
                    {"label": "📈 Croissance pop (an.)", "value": "pop_growth"},
                    {"label": "🏗️ Built-up 2025", "value": "built_2025"},
                    {"label": "🏗️ Built-up 2015", "value": "built_2015"},
                    {"label": "📈 Croissance urbaine (an.)", "value": "built_growth"},
                    {"label": "🎯 LCRPGR", "value": "lcrpgr"},
                    {"label": "🏙️ Densité 2025", "value": "density"},
                ],
                value="lcrpgr",
                clearable=False
            )
        ]),
        html.Div([
            html.Label("Filtrer entités", style={"fontWeight": "700"}),
            dcc.Dropdown(id="entity-filter-dropdown", options=[], multi=True, placeholder="Toutes")
        ])
    ]),

    html.H3("🗺️ Carte", className="title", style={"marginTop":"16px"}),
    dcc.Graph(id="main-map", style={"height":"620px"}),

    html.H3("📌 Infos au survol", className="title", style={"marginTop":"16px"}),
    html.Div(id="hover-info", style={"padding":"14px", "border":"1px solid #eee", "borderRadius":"12px"}),

    html.H3("📊 Analyses (Top 10 & relation)", className="title", style={"marginTop":"16px"}),
    html.Div(style={"display":"grid","gridTemplateColumns":"1fr 1fr","gap":"14px"}, children=[
        dcc.Graph(id="ranking-chart", style={"height":"480px"}),
        dcc.Graph(id="scatter-chart", style={"height":"480px"})
    ]),

    html.H3("📋 Tableau", className="title", style={"marginTop":"16px"}),
    html.Div(id="data-table-container"),

    html.H3("📈 Distribution LCRPGR (pixels)", className="title", style={"marginTop":"16px"}),
    dcc.Graph(id="histogram-lcrpgr", style={"height":"420px"}),

    html.Div(style={"textAlign":"center", "marginTop":"22px", "color":"#111"},
             children="Astuce : commence en mode National (raster), puis passe en Provinces/Districts/Secteurs.")
])


# ------------------------------------------------------------
# 7) CALLBACKS DASHBOARD
# ------------------------------------------------------------

@app.callback(
    Output("entity-filter-dropdown", "options"),
    Input("admin-level-dropdown", "value")
)
def update_entity_options(admin_level):
    if admin_level == "national" or admin_level not in admin_stats:
        return []
    df = admin_stats[admin_level]
    return [{"label": n, "value": n} for n in sorted(df["name"].dropna().unique())]


@app.callback(
    Output("main-map", "figure"),
    [Input("admin-level-dropdown", "value"),
     Input("indicator-dropdown", "value"),
     Input("entity-filter-dropdown", "value")]
)
def update_main_map(admin_level, indicator, selected_entities):
    # --- MODE NATIONAL: raster heatmap ---
    if admin_level == "national":
        if indicator == "pop_2015":
            data, title, cs = pop2015, "Population 2015", "YlOrRd"
        elif indicator == "pop_2025":
            data, title, cs = pop2025, "Population 2025", "YlOrRd"
        elif indicator == "pop_growth":
            data, title, cs = pop_growth, "Croissance population (an.)", "RdYlGn"
        elif indicator == "built_2015":
            data, title, cs = built2015, "Built-up 2015", "YlGnBu"
        elif indicator == "built_2025":
            data, title, cs = built2025, "Built-up 2025", "YlGnBu"
        elif indicator == "built_growth":
            data, title, cs = urban_growth, "Croissance urbaine (an.)", "Greens"
        else:
            data, title, cs = LCRPGR, "LCRPGR", "RdYlGn_r"

        fig = go.Figure(go.Heatmap(
            z=data, x=lons[0, :], y=lats[:, 0],
            colorscale=cs,
            hovertemplate="Valeur: %{z:.3f}<extra></extra>"
        ))
        fig.update_layout(title=title, margin=dict(l=0, r=0, t=30, b=0))
        return fig

    # --- MODE ADMIN: choropleth ---
    if admin_level not in admin_data or admin_level not in admin_stats:
        return go.Figure()

    gdf = admin_data[admin_level].copy()
    df = admin_stats[admin_level].copy()

    name_col = _admin_name_col(admin_level)
    gdf = gdf.merge(df, left_on=name_col, right_on="name", how="left")

    if selected_entities:
        gdf = gdf[gdf["name"].isin(selected_entities)]

    # choix de la colonne à afficher
    indicator_to_col = {
        "pop_2015": "pop_2015",
        "pop_2025": "pop_2025",
        "pop_growth": "pop_growth",
        "built_2015": "built_2015",
        "built_2025": "built_2025",
        "built_growth": "built_growth",
        "lcrpgr": "lcrpgr",
        "density": "density"
    }
    col = indicator_to_col.get(indicator, "lcrpgr")

    geojson = json.loads(gdf.to_json())
    z = gdf[col].fillna(0).values

    fig = go.Figure(go.Choroplethmapbox(
        geojson=geojson,
        locations=list(range(len(gdf))),
        z=z,
        colorscale="RdYlGn_r" if col == "lcrpgr" else "YlOrRd",
        marker_line_width=1,
        marker_line_color="white",
        text=gdf["name"],
        hovertemplate="<b>%{text}</b><br>" + f"{col}: " + "%{z:.3f}<extra></extra>"
    ))
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=7,
        mapbox_center={"lat": -1.94, "lon": 29.87},
        title=f"{col} — {admin_level}",
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig


@app.callback(
    Output("ranking-chart", "figure"),
    [Input("admin-level-dropdown", "value"),
     Input("indicator-dropdown", "value")]
)
def update_ranking(admin_level, indicator):
    if admin_level == "national" or admin_level not in admin_stats:
        return go.Figure()

    df = admin_stats[admin_level].copy()
    col = {
        "pop_2015": "pop_2015",
        "pop_2025": "pop_2025",
        "pop_growth": "pop_growth",
        "built_2015": "built_2015",
        "built_2025": "built_2025",
        "built_growth": "built_growth",
        "lcrpgr": "lcrpgr",
        "density": "density"
    }[indicator]

    df_top = df.nlargest(10, col)
    fig = go.Figure(go.Bar(
        x=df_top[col], y=df_top["name"], orientation="h",
        marker_color="#0f3a6b", text=df_top[col].round(2), textposition="outside"
    ))
    fig.update_layout(title=f"Top 10 — {col}", yaxis={'categoryorder': 'total ascending'},
                      margin=dict(l=120, r=20, t=40, b=20))
    return fig


@app.callback(
    Output("scatter-chart", "figure"),
    Input("admin-level-dropdown", "value")
)
def update_scatter(admin_level):
    if admin_level == "national" or admin_level not in admin_stats:
        return go.Figure()

    df = admin_stats[admin_level]
    fig = px.scatter(
        df, x="pop_growth", y="lcrpgr", size="pop_2025",
        color="lcrpgr", color_continuous_scale="RdYlGn_r",
        hover_name="name",
        labels={"pop_growth": "Croissance pop (an.)", "lcrpgr": "LCRPGR"}
    )
    fig.add_hline(y=1, line_dash="dash", line_color="red")
    fig.update_layout(title=f"LCRPGR vs Croissance population — {admin_level}")
    return fig


@app.callback(
    Output("data-table-container", "children"),
    [Input("admin-level-dropdown", "value"),
     Input("entity-filter-dropdown", "value")]
)
def update_table(admin_level, selected_entities):
    if admin_level == "national" or admin_level not in admin_stats:
        return html.P("Choisis un niveau administratif pour voir le tableau.")

    df = admin_stats[admin_level].copy()
    if selected_entities:
        df = df[df["name"].isin(selected_entities)]
    df = df.round(2)

    return dash_table.DataTable(
        data=df.to_dict("records"),
        columns=[{"name": c, "id": c} for c in df.columns],
        page_size=10,
        style_header={"backgroundColor": "#0f3a6b", "color": "white", "fontWeight": "bold"},
        style_cell={"padding": "8px", "textAlign": "left"},
        style_data_conditional=[{"if": {"row_index": "odd"}, "backgroundColor": "#f7f7f7"}],
        filter_action="native",
        sort_action="native"
    )


@app.callback(
    Output("hover-info", "children"),
    [Input("main-map", "hoverData"),
     Input("admin-level-dropdown", "value")]
)
def hover_info(hoverData, admin_level):
    if hoverData is None:
        return html.Div([
            html.H4("📍 Infos", style={"margin": "0 0 6px 0"}),
            html.P("Survole la carte pour voir les valeurs.")
        ])

    if admin_level == "national":
        pt = hoverData.get("points", [{}])[0]
        lon = pt.get("x"); lat = pt.get("y")
        if lon is None or lat is None:
            return html.P("Position non disponible.")
        try:
            col_idx = int((lon - transform[2]) / transform[0])
            row_idx = int((lat - transform[5]) / transform[4])
            if not (0 <= row_idx < height and 0 <= col_idx < width):
                return html.P("Hors emprise raster.")
            p = pop2025[row_idx, col_idx]
            l = LCRPGR[row_idx, col_idx]
            return html.Div([
                html.H4("📍 Pixel raster"),
                html.P(f"Lat/Lon : {lat:.4f} , {lon:.4f}"),
                html.P(f"Population 2025 : {int(p):,}" if not np.isnan(p) else "Population 2025 : NA"),
                html.P(f"LCRPGR : {l:.3f}" if not np.isnan(l) else "LCRPGR : NA")
            ])
        except Exception as e:
            return html.P(f"Erreur lecture pixel: {e}")

    # mode admin: on ne peut pas récupérer facilement 'location' via ce renderer si la carte est une heatmap
    # (on reste simple : message générique)
    return html.P("Astuce : le survol est le plus fiable en mode NATIONAL (raster).")


@app.callback(
    Output("histogram-lcrpgr", "figure"),
    Input("admin-level-dropdown", "value")
)
def update_hist(admin_level):
    vals = LCRPGR[~np.isnan(LCRPGR)]
    fig = go.Figure(go.Histogram(x=vals.flatten(), nbinsx=60, marker_color="#0f3a6b", opacity=0.8))
    fig.add_vline(x=1, line_dash="dash", line_color="red", annotation_text="LCRPGR=1")
    fig.add_vline(x=float(np.nanmedian(vals)), line_dash="dash", line_color="green",
                  annotation_text=f"médiane={np.nanmedian(vals):.2f}")
    fig.update_layout(title="Distribution LCRPGR (pixels)",
                      xaxis_title="LCRPGR", yaxis_title="Nombre de pixels")
    return fig


# ------------------------------------------------------------
# 8) LANCEMENT
# ------------------------------------------------------------
if __name__ == "__main__":
    print("\n✅ Dashboard prêt — ouvre : http://127.0.0.1:8050")
    app.run(debug=True, host="127.0.0.1", port=8050)



📂 Chargement des données...
✅ Données chargées (rasters + shapefiles).

✅ Dashboard prêt — ouvre : http://127.0.0.1:8050


C:\Users\pc\AppData\Local\Temp\ipykernel_14652\3926316087.py:437: DeprecationWarning:

*choroplethmapbox* is deprecated! Use *choroplethmap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

